## LOADING LIBRARIES AND DATA

In [1]:
import pandas as pd
import numpy as np
import copy
import json
from numpy import mean
from numpy import std
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [2]:
intersection_df = pd.read_json('intersection_df.json',  orient="records", lines = True)

In [3]:
intersection_df.head()

,text,label,label_new,binary,label_binary,label_new_2
0,I'm sure most people probably figured this is ...,Catastrophizing,Catastrophizing,1,Distorted,Catastrophizing
1,I've started to realise my nausea is mostly ca...,Not Distorted,Not Distorted,0,Not Distorted,Not Distorted
2,Every single day that I get my work done witho...,Not Distorted,Not Distorted,0,Not Distorted,Not Distorted
3,For a long long time I have found it difficult...,Not Distorted,Not Distorted,0,Not Distorted,Not Distorted
4,I’m sorry. I know it hurts. I know the pain of...,Not Distorted,Not Distorted,0,Not Distorted,Not Distorted


## BINARY LOGISTIC REGRESSION WITH NESTED STRATIFIED KFOLD CROSS-VALIDATION

In [4]:
# create dataset
X = intersection_df['text']
y = intersection_df['binary']

In [5]:
#source: https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/?fbclid=IwAR0H80UxAb7r6cVgD2c5kAfqthfYBWOupzLwRk1Mg7l8BDcR9_EcASHHCsc
#source: https://www.geeksforgeeks.org/stratified-k-fold-cross-validation/

pipeline = Pipeline([
                  ('vect', CountVectorizer(max_features=5000,min_df=10)), 
                  ('tfidf', TfidfTransformer()), 
                  ('logreg', LogisticRegression(solver='liblinear', random_state=1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X,y):
  # split data
  X_train, X_test = X[train_ix], X[test_ix]
  y_train, y_test = y[train_ix], y[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['logreg__C'] = [0.01,0.03,0.05,0.1,0.3,0.5,1,3,5]
  #space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  #space['vect__min_df'] = [1,3,5,10]
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.809, est=0.654, cfg={'logreg__C': 5}
              precision    recall  f1-score   support

           0       0.82      0.93      0.87        15
           1       0.80      0.57      0.67         7

    accuracy                           0.82        22
   macro avg       0.81      0.75      0.77        22
weighted avg       0.82      0.82      0.81        22

>f1score=0.752, est=0.693, cfg={'logreg__C': 5}
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        15
           1       0.75      0.43      0.55         7

    accuracy                           0.77        22
   macro avg       0.76      0.68      0.70        22
weighted avg       0.77      0.77      0.75        22

>f1score=0.616, est=0.660, cfg={'logreg__C': 3}
              precision    recall  f1-score   support

           0       0.70      0.93      0.80        15
           1       0.50      0.14      0.22         7

    accuracy                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.590, est=0.690, cfg={'logreg__C': 5}
              precision    recall  f1-score   support

           0       0.71      0.94      0.81        16
           1       0.00      0.00      0.00         6

    accuracy                           0.68        22
   macro avg       0.36      0.47      0.41        22
weighted avg       0.52      0.68      0.59        22

>f1score=0.733, est=0.649, cfg={'logreg__C': 5}
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        15
           1       0.67      0.33      0.44         6

    accuracy                           0.76        21
   macro avg       0.72      0.63      0.65        21
weighted avg       0.75      0.76      0.73        21

>f1score=0.733, est=0.685, cfg={'logreg__C': 5}
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        15
           1       0.67      0.33      0.44         6

    accuracy                          

# MULTI-CLASS LOGISTIC REGRESSION

### Catastrophizing vs Other distortion vs Labeling

In [6]:
intersection_df['label_new'].value_counts()

Not Distorted       152
Catastrophizing      43
Other distortion     12
Labeling              9
Name: label_new, dtype: int64

In [7]:
df_with_distortions = intersection_df[intersection_df['binary'] == 1]
df_with_distortions = df_with_distortions.reset_index(drop=True)
df_with_distortions['label_new'].value_counts()

Catastrophizing     43
Other distortion    12
Labeling             9
Name: label_new, dtype: int64

In [8]:
X_multi = df_with_distortions['text']
y_multi = df_with_distortions['label_new']

In [9]:
pipeline = Pipeline([
                  ('vect', CountVectorizer(max_features=5000,min_df=10)), 
                  ('tfidf', TfidfTransformer()), 
                  ('logreg', LogisticRegression(solver='liblinear', random_state=1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=9, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X_multi,y_multi):
  # split data
  X_train, X_test = X_multi[train_ix], X_multi[test_ix]
  y_train, y_test = y_multi[train_ix], y_multi[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['logreg__C'] = [0.01,0.03,0.05,0.1,0.3,0.5,1,3,5]
  #space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  #space['vect__min_df'] = [1,3,5,10]
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.687, est=0.585, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.71      1.00      0.83         5
        Labeling       0.00      0.00      0.00         1
Other distortion       1.00      0.50      0.67         2

        accuracy                           0.75         8
       macro avg       0.57      0.50      0.50         8
    weighted avg       0.70      0.75      0.69         8



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.595, est=0.534, cfg={'logreg__C': 0.01}
                  precision    recall  f1-score   support

 Catastrophizing       0.71      1.00      0.83         5
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         1

        accuracy                           0.71         7
       macro avg       0.24      0.33      0.28         7
    weighted avg       0.51      0.71      0.60         7



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.595, est=0.540, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.71      1.00      0.83         5
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         1

        accuracy                           0.71         7
       macro avg       0.24      0.33      0.28         7
    weighted avg       0.51      0.71      0.60         7



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.416, est=0.582, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.57      1.00      0.73         4
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.57         7
       macro avg       0.19      0.33      0.24         7
    weighted avg       0.33      0.57      0.42         7



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.416, est=0.590, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.57      1.00      0.73         4
        Labeling       0.00      0.00      0.00         1
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.57         7
       macro avg       0.19      0.33      0.24         7
    weighted avg       0.33      0.57      0.42         7



F1-score weighted Mean and Std: 0.566 (0.085)
Max F1-score weighted: 0.6874999999999999


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Catastrophizing vs Other vs Not distorted

In [10]:
intersection_df['label_new_2'].value_counts()

Not Distorted       152
Catastrophizing      43
Other distortion     21
Name: label_new_2, dtype: int64

In [11]:
X_multi = intersection_df['text']
y_multi = intersection_df['label_new_2']

In [12]:
pipeline = Pipeline([
                  ('vect', CountVectorizer(max_features=5000,min_df=10)), 
                  ('tfidf', TfidfTransformer()), 
                  ('logreg', LogisticRegression(solver='liblinear', random_state=1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X_multi,y_multi):
  # split data
  X_train, X_test = X_multi[train_ix], X_multi[test_ix]
  y_train, y_test = y_multi[train_ix], y_multi[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['logreg__C'] = [0.01,0.03,0.05,0.1,0.3,0.5,1,3,5]
  #space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  #space['vect__min_df'] = [1,3,5,10]
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.553, est=0.595, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         5
   Not Distorted       0.68      1.00      0.81        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.68        22
       macro avg       0.23      0.33      0.27        22
    weighted avg       0.46      0.68      0.55        22



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.553, est=0.628, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         5
   Not Distorted       0.68      1.00      0.81        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.68        22
       macro avg       0.23      0.33      0.27        22
    weighted avg       0.46      0.68      0.55        22



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.568, est=0.612, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         5
   Not Distorted       0.71      1.00      0.83        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.68        22
       macro avg       0.24      0.33      0.28        22
    weighted avg       0.49      0.68      0.57        22



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.546, est=0.600, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         4
   Not Distorted       0.68      0.87      0.76        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.62        21
       macro avg       0.23      0.29      0.25        21
    weighted avg       0.49      0.62      0.55        21



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.595, est=0.609, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.00      0.00      0.00         4
   Not Distorted       0.71      1.00      0.83        15
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.71        21
       macro avg       0.24      0.33      0.28        21
    weighted avg       0.51      0.71      0.60        21



F1-score weighted Mean and Std: 0.593 (0.063)
Max F1-score weighted: 0.7676767676767677


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# BINARY - catastrophizing vs other

In [13]:
X_multi = df_with_distortions['text']
y_multi = df_with_distortions['label_new_2']

In [14]:
y_multi.value_counts()

Catastrophizing     43
Other distortion    21
Name: label_new_2, dtype: int64

In [15]:
pipeline = Pipeline([
                  ('vect', CountVectorizer(max_features=5000,min_df=10)), 
                  ('tfidf', TfidfTransformer()), 
                  ('logreg', LogisticRegression(solver='liblinear', random_state=1))
                  ])

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X_multi,y_multi):
  # split data
  X_train, X_test = X_multi[train_ix], X_multi[test_ix]
  y_train, y_test = y_multi[train_ix], y_multi[test_ix]
  # configure the cross-validation procedure
  cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
  # define the model
  model = pipeline
  # define search space
  space = dict()
  space['logreg__C'] = [0.01,0.03,0.05,0.1,0.3,0.5,1,3,5]
  #space['vect__max_features'] = [5000,4000,3000,2000,1000,500,5]
  #space['vect__min_df'] = [1,3,5,10]
  # define search
  search = GridSearchCV(model, space, scoring='f1_weighted', cv=cv_inner, refit=True)
  # execute search
  result = search.fit(X_train, y_train)
  # get the best performing model fit on the whole training set
  best_model = result.best_estimator_
  # evaluate model on the hold out dataset
  yhat = best_model.predict(X_test)
  # evaluate the model
  f1 = f1_score(y_test, yhat, average='weighted')
  # store the result
  outer_results.append(f1)
  # report progress
  print('>f1score=%.3f, est=%.3f, cfg=%s' % (f1, result.best_score_, result.best_params_))
  print(classification_report(y_test, yhat))
# summarize the estimated performance of the model
print('\n')
print('F1-score weighted Mean and Std: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Max F1-score weighted: ' + str(max(outer_results)))

>f1score=0.429, est=0.589, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.60      0.60      0.60         5
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.43         7
       macro avg       0.30      0.30      0.30         7
    weighted avg       0.43      0.43      0.43         7

>f1score=0.595, est=0.594, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.71      1.00      0.83         5
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.71         7
       macro avg       0.36      0.50      0.42         7
    weighted avg       0.51      0.71      0.60         7



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.840, est=0.563, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.83      1.00      0.91         5
Other distortion       1.00      0.50      0.67         2

        accuracy                           0.86         7
       macro avg       0.92      0.75      0.79         7
    weighted avg       0.88      0.86      0.84         7

>f1score=0.416, est=0.628, cfg={'logreg__C': 3}
                  precision    recall  f1-score   support

 Catastrophizing       0.57      1.00      0.73         4
Other distortion       0.00      0.00      0.00         3

        accuracy                           0.57         7
       macro avg       0.29      0.50      0.36         7
    weighted avg       0.33      0.57      0.42         7



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.815, est=0.564, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.80      1.00      0.89         4
Other distortion       1.00      0.50      0.67         2

        accuracy                           0.83         6
       macro avg       0.90      0.75      0.78         6
    weighted avg       0.87      0.83      0.81         6

>f1score=0.533, est=0.588, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.67      1.00      0.80         4
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.67         6
       macro avg       0.33      0.50      0.40         6
    weighted avg       0.44      0.67      0.53         6



...

>f1score=0.444, est=0.656, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.60      0.75      0.67         4
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.50         6
       macro avg       0.30      0.38      0.33         6
    weighted avg       0.40      0.50      0.44         6

>f1score=0.533, est=0.594, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.67      1.00      0.80         4
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.67         6
       macro avg       0.33      0.50      0.40         6
    weighted avg       0.44      0.67      0.53         6



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.533, est=0.569, cfg={'logreg__C': 3}
                  precision    recall  f1-score   support

 Catastrophizing       0.67      1.00      0.80         4
Other distortion       0.00      0.00      0.00         2

        accuracy                           0.67         6
       macro avg       0.33      0.50      0.40         6
    weighted avg       0.44      0.67      0.53         6



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>f1score=0.667, est=0.600, cfg={'logreg__C': 5}
                  precision    recall  f1-score   support

 Catastrophizing       0.75      0.75      0.75         4
Other distortion       0.50      0.50      0.50         2

        accuracy                           0.67         6
       macro avg       0.62      0.62      0.62         6
    weighted avg       0.67      0.67      0.67         6



F1-score weighted Mean and Std: 0.581 (0.143)
Max F1-score weighted: 0.8398268398268397
